# Approx Lab

The goal of this lab is to gain experience using approximation algorithms.

For the purpose of this assignment, you will be implementing the knapsack problem. Before continuing, the following cell contains any dependencies you may need for this assignment.

In [1]:
import pandas as pd

## Background Info

### Knapsack Problem
We are actually going to be looking at a specific (hopefully fun) implementation of the knapsack problem.

The traditional knapsack problem asks you to pack a set of items, with given weights and values into a "knapsack" with a maximum capacity. You cannot pack more items than the capacity of the knapsack. The goal is to select a subset of items that maximize the total value with what can fit in the knapsack. The name "knapsack" comes from the similarity to a hiker trying to pack a bag without going over a certain weight.

The knapsack problem is an NP-Hard optimization problem, which is why it is one of the classic approximation algorithms.

### Specifications for this Lab
The variation that we will be doing in this assignment is referred to as an 0/1 knapsack problem, meaning that items must be completely put in the knapsack or they must be left behind (you can't have a fractional portion of an item). We will be representing this as a simple fantasy sports league - specifically Formula 1 drivers. 

You are provided with a list of F1 drivers and constructors with given costs and points as well as a budget of $100M. You will be selecting five drivers and two constructors (teams). 

### Scoring Information
Points are gained from qualification results for drivers and constructors as well as race results for drivers and constructors. 

<!-- Qualifications
- Drivers score based on their resulting position 
- Constructor score the combined total of their two drivers in qualifying plus a modifier based on how far both their drivers get. -->
<!-- 
| Drivers    | Points |
| -------- | ------- |
| Pole Position (1st Place) | 10 |
| 2nd Place | 9 |
| 3rd Place | 8 |
| 4th Place | 7 |
| 5th Place | 6 |
| 6th Place | 5 |
| 7th Place | 4 |
| 8th Place | 3 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| No Time Set | 0 |
| Disqualification | 0 | -->


<!-- | Constructors | Points |
| ----------- | ------- |
| Neither Driver Reaches Q2 | -1 |
| One Driver Reaches Q2 | 1 |
| Both Drivers Reach Q2 | 5 |
| Both Drivers Reach Q3 | 10 | -->


<!-- Race
- Drivers score based on their resulting position, position difference from starting order, and some additional bonuses
- Constructors score a combined total of their two drivers' results (excluding the Driver of the Day bonus) ~~plus a modifier based on pitstop times~~ -->

<!-- | Drivers | Points |
| -------- | ------- |
| Positions Gained | 1/position |
| Positions Lost | -1/position |
| Overtakes Made | 1/overtake |
| Fastest Lap | 10 |
| Driver of the Day | 10 | -->

<!-- | Race Result | Points |
| -------- | ------- |
| 1st Place | 25 |
| 2nd Place | 18 |
| 3rd Place | 15 |
| 4th Place | 12 |
| 5th Place | 10 |
| 6th Place | 8 |
| 7th Place | 6 |
| 8th Place | 4 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| DNF | -20 |
| Disqualification | -20 | -->

Fortunately for you, we have access to the current point standings post-2025 China GP for drivers & constructors, which are available in `drivers.csv` & `constructors.csv`.

*Note: The costs are in millions, so 30 is actually 30 M.

### Questions (Answered in Various Parts of the Lab): 
1. What is the general setup of the problem?
2. Determining the mathematical functions of the problem
3. Implementing a greedy algorithm


## A. Assessing the Problem

The values that are provided for this lab are from the points distribution PRIOR to the Japan Grand Prix (which means we can actually compare with the race results at the submission deadline!)

Create a fantasy team https://fantasy.formula1.com/en/create-team for the upcoming Grand Prix to get a better idea of what drivers, constructors, points, and the budget are. Who is on your team & how much did you spend? Don't spend too much time on this, it really is just to get familiar with the format.

Drivers: M. Verstappen, L. Norris, Y. Tsnoda
Constructors: Mercedes
What represents the 
Total Spening: $99.1M
"items" in the knapsack? 

drivers and constructors
What represents the weights?
costs of drivers and constructors

What represents the values?

the expected points from the drivers and constructors

In [5]:
# Load the data from the CSV files into a format that you are comfortable working with
# This can be arrays, dictionaries, or pd DataFrames

# Print/Display your data as well to confirm that you have successfully saved it.
import pandas as pd

# Load data
drivers = pd.read_csv("drivers.csv")
constructors = pd.read_csv("constructors.csv")

# Display data
print("Drivers Data:")
print(drivers.head())

print("\nConstructors Data:")
print(constructors.head())


Drivers Data:
                  Driver  Points  Cost
0           Lando Norris     100  29.6
1  Andrea Kimi Antonelli      61  19.3
2         George Russell      60  21.4
3         Max Verstappen      59  28.6
4          Oscar Piastri      55  23.0

Constructors Data:
       Constructor  Points  Cost
0          McLaren     172  30.6
1         Mercedes     136  23.3
2  Red Bull Racing      86  25.4
3             Haas      61   8.2
4         Williams      36  13.5


# B. Integer Program

Before implementing our approx algorithm, we need to first consider the integer program. 

Given the following variables, create an integer program that maximizes the total points of drivers while remaining under the budget: 
- $x_i$ is the decision variable that determines whether driver $i$ is included in the team or not
- $p_i$ is the # of points that driver $i$ is worth
- $c_i$ is the cost of driver $i$

For now, we will focus on just the drivers and ignore the constructors. We will also temporarily reduce our budget to **$50M**

While we are not planning on implementing the full integer program, it helps us quantify parts of the problem


$\text{maximize} \quad \sum_{i=1}^{n} x_i \cdot p_i$ \
$\text{subject to} \quad \sum_{i=1}^{n} x_i \cdot c_i \leq 50$ \
$\sum_{i=1}^{n} x_i \leq 5$ \
$x_i \in {0, 1}$ for all $i$


## Decision Variable & Constraints

We can make helper functions for our future algorithm based on constraints that we derive when forming the IP: 
1. Which drivers should be included in our solution? 
2. What is our maximum cost?
3. What is our maximum # of drivers? 

In [8]:
budget = 50 

# Set temporary budget for drivers only
max_drivers = 5

# Filter eligible drivers under budget
def eligible_drivers(drivers_df, budget):
    return drivers_df[drivers_df["Cost"] <= budget]

def get_max_cost():
    return budget

def get_max_drivers():
    return max_drivers

# Display eligible drivers
print("Eligible Drivers (Cost ≤ 50M):")
print(eligible_drivers(drivers, budget))


Eligible Drivers (Cost ≤ 50M):
                   Driver  Points  Cost
0            Lando Norris     100  29.6
1   Andrea Kimi Antonelli      61  19.3
2          George Russell      60  21.4
3          Max Verstappen      59  28.6
4           Oscar Piastri      55  23.0
5            Lance Stroll      33   9.3
6            Esteban Ocon      32   8.1
7          Alexaner Albon      28  12.8
8         Nico Hulkenberg      22   7.6
9          Oliver Bearman      22   6.7
10         Lewis Hamilton       3  23.6
11           Yuki Tsunoda       0  16.8
12            Liam Lawson       0   8.4
13        Charles LeClerc       0  25.3
14           Pierre Gasly      -7  10.6
15           Isack Hadjar      -9   5.0
16            Jack Doohan      -9   6.0
17           Carlos Sainz     -11  11.9
18      Gabriel Bortoleto     -11   4.8
19        Fernando Alonso     -36   7.6


# C. Greedy Algorithm


There are multiple ways we can go about solving the knapsack problem. For this lab, we will be focusing on a greedy algorithm and a variant of that.

Let's start with a basic greedy algorithm:
1. Calculate the value-to-cost ratio for each driver 
2. Sort drivers in non-increasing order of value-to-cost
2. Greedily select drivers from list until either budget or max drivers constraint is hit

Use the helper methods from the previous part in your implementation.

In [9]:
def value_to_cost(drivers_df): 
    df = drivers_df.copy()
    df["Ratio"] = df["Points"] / df["Cost"]
    return df

def sort_items(drivers_df): 
    return drivers_df.sort_values(by="Ratio", ascending=False).reset_index(drop=True)

def greedy_driver_select(drivers_df):
    df = sort_items(value_to_cost(drivers_df))
    
    selected = []
    total_cost = 0

    for _, row in df.iterrows():
        if len(selected) < max_drivers and total_cost + row["Cost"] <= budget:
            selected.append(row)
            total_cost += row["Cost"]
    
    return pd.DataFrame(selected)

In [10]:
selected_drivers = greedy_driver_select(drivers)
print("Selected Drivers:")
print(selected_drivers)

print(f"\nTotal Cost: {selected_drivers['Cost'].sum()}M")
print(f"Total Points: {selected_drivers['Points'].sum()}")

Selected Drivers:
         Driver  Points  Cost     Ratio
0  Esteban Ocon      32   8.1  3.950617
1  Lance Stroll      33   9.3  3.548387
2  Lando Norris     100  29.6  3.378378

Total Cost: 47.0M
Total Points: 165


Try running your basic greedy algorithm. 

You may notice a few flaws:
1. The greedy algorithm is only looking at value-to-cost ratio, which may miss out on an overall improvement in value that still fits within the budget
2. Depending on your implementation, you may not currently be meeting the "5 total drivers" constraint - this is fine


## Redux Greedy Algorithm
We are going to next implement a small adjustment to this algorithm in order to improve its performance. \
This is a 2-approximation for the knapsack problem. 

1. Calculate the value-to-cost ratio for each driver
2. Sort drivers in non-increasing order of value-to-cost
3. Greedily add items until we hit an item that is too big
4. Compare the too big item against the already existing list of items & pick the better of the two

It might help to make a separate function to handle #4. 

In [11]:
def redux_greedy_driver_select(drivers_df, budget=50, max_drivers=5):
    df = sort_items(value_to_cost(drivers_df))
    
    selected = []
    total_cost = 0
    total_points = 0

    # Greedy selection
    for _, row in df.iterrows():
        if len(selected) < max_drivers and total_cost + row["Cost"] <= budget:
            selected.append(row)
            total_cost += row["Cost"]
            total_points += row["Points"]
        else:
            too_big_item = row
            break  

    greedy_df = pd.DataFrame(selected)
    greedy_points = total_points

    if too_big_item["Cost"] <= budget:
        if too_big_item["Points"] > greedy_points:
            return pd.DataFrame([too_big_item])
    
    return greedy_df

redux_selected = redux_greedy_driver_select(drivers)
print("Redux Greedy Selected Drivers:")
print(redux_selected)

print(f"\nTotal Cost: {redux_selected['Cost'].sum()}M")
print(f"Total Points: {redux_selected['Points'].sum()}")


Redux Greedy Selected Drivers:
         Driver  Points  Cost     Ratio
0  Esteban Ocon      32   8.1  3.950617
1  Lance Stroll      33   9.3  3.548387
2  Lando Norris     100  29.6  3.378378

Total Cost: 47.0M
Total Points: 165


# D. Reintroducing Constructors

We've been neglecting our constructors so far and cutting our budget. Let's make them feel included in our processes again. 

## Integer Program

We should start by redefining our original IP to see what constraints we need to add. \
We've added some more variables to our IP: 
- $x_j$ is the decision variable that determines whether constructor $j$ is included in the team or not 
- $p_j$ is the # of points that constructor $j$ is worth
- $c_j$ is the cost of constructor $j$

Our budget has also been increased back to **100M**. 

$\text{maximize} \quad \sum_{i=1}^{n} x_i \cdot pp_i + \sum_{j=1}^{m} y_j \cdot pp_j$ \
$\text{subject to} \quad \sum_{i=1}^{n} x_i \cdot cc_i + \sum_{j=1}^{m} y_j \cdot cc_j \leq 100$ \
$\sum_{i=1}^{n} x_i = 5$ \
$\sum_{j=1}^{m} y_j = 2$ \
$x_i, y_j \in {0, 1}$


## Constraints 

To redefine our greedy algorithm to include constructors, we have to also write helpers that correspond to our new constraints.

In [13]:
budget = 100 
max_drivers = 5
max_teams = 2

## Greedy Algorithm

Update the greedy algorithm to calculate for buying both constructors and drivers. \
Remember, your list of non-increasing value-to-cost ratios should be a combination of both drivers & constructors. 

In [ ]:
def greedy_f1(drivers_df, constructors_df):
    # Tag item types
    drivers = drivers_df.copy()
    drivers["Type"] = "Driver"
    
    constructors = constructors_df.copy()
    constructors["Type"] = "Constructor"

    # Combine and calculate value-to-cost ratio
    combined = pd.concat([drivers, constructors], ignore_index=True)
    combined["Ratio"] = combined["Points"] / combined["Cost"]
    combined = combined.sort_values(by="Ratio", ascending=False).reset_index(drop=True)

    selected = []
    total_cost = 0
    driver_count = 0
    constructor_count = 0

    for _, row in combined.iterrows():
        cost = row["Cost"]
        if total_cost + cost > budget:
            continue
        if row["Type"] == "Driver" and driver_count < max_drivers:
            selected.append(row)
            total_cost += cost
            driver_count += 1
        elif row["Type"] == "Constructor" and constructor_count < max_teams:
            selected.append(row)
            total_cost += cost
            constructor_count += 1

        if driver_count == max_drivers and constructor_count == max_teams:
            break

    return pd.DataFrame(selected)

# Run it
team = greedy_f1(drivers, constructors)

print("Greedy F1 Fantasy Team:")
print(team)

print(f"\nTotal Cost: {team['Cost'].sum()}M")
print(f"Total Points: {team['Points'].sum()}")


Greedy F1 Fantasy Team:
             Driver  Points  Cost         Type Constructor     Ratio
0               NaN      61   8.2  Constructor        Haas  7.439024
1               NaN     136  23.3  Constructor    Mercedes  5.836910
4      Esteban Ocon      32   8.1       Driver         NaN  3.950617
5      Lance Stroll      33   9.3       Driver         NaN  3.548387
7      Lando Norris     100  29.6       Driver         NaN  3.378378
8    Oliver Bearman      22   6.7       Driver         NaN  3.283582
10  Nico Hulkenberg      22   7.6       Driver         NaN  2.894737

Total Cost: 92.8M
Total Points: 406
